### Importy, dekorator do mierzenia czasu oraz podstawowa data używana przy odczytywaniu danych

In [2]:
from datetime import datetime, timedelta
import csv
from math import radians, sin, cos, atan2, sqrt
from collections import namedtuple
from itertools import count
from heapq import heappush, heappop
import sys

from timeit import default_timer as timer

def timer_func(func):
    def wrapper(*args, **kwargs):
        t1 = timer()
        result = func(*args, **kwargs)
        t2 = timer()
        print(f'{func.__name__}() executed in {(t2-t1):.6f}s\n')
        return result
    return wrapper

base_date = datetime.strptime('2024-03-08', '%Y-%m-%d')

---

# Moja struktura grafu skierowanego

Struktura składa się z trzech głównych klas:

1. **Node**: Reprezentuje węzeł grafu, który jest przystankiem komunikacyjnym. Posiada nazwę, szerokość geograficzną, długość geograficzną oraz słownik krawędzi, które łączą ten węzeł z innymi węzłami. Metoda `add_edge` dodaje nową krawędź do węzła.

2. **Edge**: Reprezentuje krawędź grafu, która łączy dwa węzły (przystanki komunikacyjne). Zawiera informacje o czasie podróży, odległości, linii komunikacyjnej itp. Krawędź zawierać różne czasy podróży w liście jako krotki, ponieważ istnieje kilka takich samych kursów, ale o innej godzinie. Zaimplementowane są metody, takie jak `add_travel_time` oraz `find_fastest_departure`, które pomagają w znajdowaniu najszybszego terminu podróży.

3. **ConnectionGraph**: Reprezentuje graf połączeń komunikacyjnych. Wczytuje dane z pliku CSV, tworzy węzły i krawędzie, a następnie wyświetla informacje o utworzonym grafie. W pliku `connection_graph.csv` istnieje kilkadziesiąt przystanków o takiej samej nazwie, różniącej się jedynie koordynatami. Zależało mi bardzo na wykorzystaniu słowników, dlatego wybrałem drogę, aby brać pod uwagę tylko pierwszy przystanek o danej nazwie, a resztę odrzucać. Oprócz tego isnieje kilka połączeń od tego przystanku o tej samej nazwie do przystanku również o tej nazwie. Przez moje wcześniej opisane podejście powodowało to pewne komplikacje związane z liczeniem odległości pomiędzy przystankami, z tego powodu postanowiłem również nie brać takich połączeń pod uwagę. Dodatkowo w celach czysto statystycznych przetrzymuję wszystkie krawędzie w liście `edges`.

Cała implementacja jest oparta na słownikach, co pozwala na efektywne przeszukiwanie grafu i operacje na węzłach oraz krawędziach. Dodatkowo, kod zawiera funkcję `parse_custom_time`, która parsuje niestandardowe formaty czasu (godziny nocne, a ponieważ korzystam z obiektów datetime zamieniam ja na normalne godziny nocne kolejnego dnia) oraz funkcję `distance`, która oblicza odległość między dwoma węzłami na sferze ziemskiej przy użyciu wzoru Haversine'a, który swoją drogą zapewnia bardzo zadowalające rezultaty. Wyniki te są w metrach.

Po krótce: posiadam graf, który posiada węzły w słowniku, a każdy węzeł posiada krawędzie również w słowniku, prowadzące do innych węzłów itd.

In [3]:
class Node:
    def __init__(self, name, lat, lon):
        self.name = name
        self.lat = lat
        self.lon = lon
        self.edges = {}

    def __str__(self) -> str:
        return f'{self.name}: {self.lat}, {self.lon}, {len(self.edges)} połączeń.'
    
    def add_edge(self, edge):
        if edge.start_node == self:
            if edge.end_node in self.edges:
                self.edges[edge.end_node].add_travel_time(edge)
            else:
                self.edges[edge.end_node] = edge

class Edge:
    def __init__(self, start_node : Node, end_node : Node, departure_time, arrival_time, line):
        self.start_node : Node = start_node
        self.end_node : Node = end_node
        self.travel_times = [(departure_time, arrival_time)]
        self.distance = distance(self.start_node, self.end_node)
        self.line = line

    def __str__(self):
        travel_info = f"    -> {self.end_node.name}:\n"
        for departure_time, arrival_time in self.travel_times:
            travel_info += f"        ({departure_time.strftime('%H:%M:%S')}) -> ({arrival_time.strftime('%H:%M:%S')}) - linia {self.line}\n"
        return travel_info.strip()
    
    def add_travel_time(self, departure_time : datetime, arrival_time: datetime):
        self.travel_times.append((departure_time, arrival_time))

    def add_travel_time(self, edge):
        self.travel_times.append((edge.travel_times[0]))
    
    def find_fastest_departure(self, target_time):
        fastest_departure = min((x for x in self.travel_times if x[0] > target_time), key=lambda x: abs(x[0] - target_time), default=None)
        return fastest_departure
    
    def find_fastest_departure_index(self, target_time):
        closest_index = None
        closest_time_difference = None

        for i, (departure_time, _) in enumerate(self.travel_times):
            if departure_time <= target_time:
                departure_time += timedelta(days=1)
                
            time_difference = abs(departure_time - target_time)
            if closest_time_difference is None or time_difference < closest_time_difference:
                closest_index = i
                closest_time_difference = time_difference
                

        return closest_index
    
    def find_nearest_departure_index(self, target_time):
        filtered_travel_times = [(i, x) for i, x in enumerate(self.travel_times) if x[0] > target_time]
        if not filtered_travel_times:
            return None
        nearest_departure_index = min(filtered_travel_times, key=lambda x: abs(x[1][0] - target_time))[0]
        return nearest_departure_index
    
    def find_nearest_departure_index2(self, target_time):
        filtered_travel_times = [(i, x) for i, x in enumerate(self.travel_times)]
        # Dodanie 24 godzin do czasów wyjazdu, które są mniejsze niż podana godzina
        filtered_travel_times = [(i, (dep + timedelta(days=1), arr)) if dep < target_time else (i, (dep, arr)) for i, (dep, arr) in filtered_travel_times]
        nearest_departure_index = min(filtered_travel_times, key=lambda x: abs(x[1][0] - target_time))[0]
        return nearest_departure_index

class ConnectionGraph:
    def __init__(self, filename):
        self.nodes = {}
        self.edges = []

        with open(filename, 'r', encoding="utf8") as csvfile:
            reader = csv.reader(csvfile, delimiter=",")
            next(reader, None)

            for row in reader:
                _, _, line, _, _, start_stop, end_stop, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon = row

                if start_stop not in self.nodes:
                    self.nodes[start_stop] = Node(start_stop, float(start_stop_lat), float(start_stop_lon))
                if end_stop not in self.nodes:
                    self.nodes[end_stop] = Node(end_stop, float(end_stop_lat), float(end_stop_lon))

                parsed_departure = parse_custom_time(row[3])
                parsed_arrival = parse_custom_time(row[4])

                # Utworzenie krawędzi
                new_edge = Edge(
                    self.nodes[start_stop],
                    self.nodes[end_stop],
                    parsed_departure,
                    parsed_arrival,
                    line
                )

                if(start_stop != end_stop):
                    self.edges.append(new_edge)
                    self.nodes[start_stop].add_edge(new_edge)

        # Wyświetlenie informacji o grafie
        print(f"Utworzono graf z pliku {filename}:")
        print(f"Liczba węzłów: {len(self.nodes)}")
        print(f"Liczba krawędzi: {len(self.edges)}")

def parse_custom_time(time_str):
    hour, minute, second = map(int, time_str.split(':'))
    if hour >= 24:
        hour -= 24
        return base_date + timedelta(days=1, hours=hour, minutes=minute, seconds=second)
    else:
        return base_date + timedelta(hours=hour, minutes=minute, seconds=second)
    
def distance(start_node : Node, end_node : Node):

    start_lat = start_node.lat
    start_lon = start_node.lon
    end_lat =  end_node.lat
    end_lon = end_node.lon

    lat1 = radians(start_lat)
    lat2 = radians(end_lat)

    # Wzór Haversine'a
    dlat = radians(end_lat - start_lat)
    dlon = radians(end_lon - start_lon)

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))


    # Promień Ziemi w m
    earth_radius = 6371e3

    return c * earth_radius

## Odczytanie danych

Utworzony graf przypisujemy do zmiennej `graph`. Samo jego tworzenie trwa zaledwie 7s, co wydaje się być dobrym wynikiem. Możemy zauważyć, że przechowujemy 939 węzłów, a także aż 982847 krawędzi.

In [4]:
graph = ConnectionGraph('connection_graph.csv')

Utworzono graf z pliku connection_graph.csv:
Liczba węzłów: 939
Liczba krawędzi: 982847


### Sprawdzenie danych
Jeżeli chodzi o `odległość` między przystankami to postanowiłem użyć metrów jako głównej jednostki, co również wpłynie na późniejsze funkcje heurystyczne.

In [5]:
distance(start_node=graph.nodes['Armii Krajowej'], end_node=graph.nodes['AUCHAN'])

7699.689400956747

In [6]:
print(graph.nodes["8 Maja"])
print(graph.edges[962000])

8 Maja: 51.11366919, 17.09111971, 4 połączeń.
-> Wilkszyn - Miłoszyn:
        (13:34:00) -> (13:35:00) - linia 923


---

# Implementacja algorytmu

Postanowiłem napisać jedną metodę do wszystkiego - tzn. posiada ona możliwość uruchomienia algorytmu Dijkstry w oparciu o kryterium czasu lub przesiadek, a także A* poprzez dodanie w parametrze funkcji heurystycznej. Dzięki temu metoda ta jest bardzo elastyczna i pozwala na poszukiwanie najbardziej optymalnych parametrów. Jego działanie jest dość proste i dzięki wykorzystaniu w moim grafie słowników dość wydajne. Struktura algorytmu została opisana poniżej w zakomentowanych linijkach. Do funkcji podpiąłem również wcześniej utworzony dekorator, który umożliwi dokładne liczenie czasu wykonywania.

Problematyczne okazało się przechowywanie w każdym `Node` listy różnych odjazdów - przez to byłem zmuszony zapisywać za każdym razem również indeks konkretnej krotki z czasami jazdy. Być może wpłynęło to trochę na wydajność, ale również dzięki temu słownik przystanków nie jest ogromny. Oprócz tego zamiast wyszukiwać i sprawdzać każdego połączenia o każdej godzinie postanowiłem napisać funkcję `find_nearest_departure_index2`, która znajduje się w klasie krawędzi. Pozwala ona pobrać najbliższy odjazd dla podanej godziny, dzięki temu biorę pod uwagę jedynie jeden termin odjazdu i oszczędzam czas na sprawdzaniu pozostałych. Funkcja ta również w przypadku, gdy nie odnajdzie najbliższego odjazdu (ze względu na dane w `connection_graph` i ograniczeniu na jeden dzień) dodaje do każego odjazdu 24h, dzięki czmeu możemy brać pod uwagę odjazdy z kolejnego dnia.

Ogólny zarys algorytmu zaczerpnąłem z biblioteki [`Dijkstar`](https://pypi.org/project/Dijkstar/), który następnie przystosowałem pod moje wymagania i strukturę moich danych - co spowodowało pełną modyfikację tejże biblioteki. Było to spowodowane różnymi godzinami różnych przystanków i krawędzi oraz generalnej struktury grafu.

In [7]:
@timer_func
def shortest_path_algorithm(
        graph : ConnectionGraph, s : Node, d : Node, start_time, cost_fun = None, cost_fun2 = None, cost_fun3 = None, heuristic_func=None):
    
    counter = count()

    # Słownik aktualnie poznanych kosztów od s do wszystkich przystanków, do których póki co dotarliśmy.
    costs = {s: 0}

    #    ReachedNode          Edge         Cost
    #              |  PrevNode |  Index     |
    #              |    |      |     |      |
    #              \/   \/     \/    \/    \/
    predecessors = {s: (None, None, None, None)}

    # Kolejka priorytetowa przystanków, które poznaliśmy, ze znanym kosztem, ale których jeszcze nie odwiedziliśmy.
    visit_queue = [(0, next(counter), s)]

    # Odzwiedzone przystanki, czyli takie, które zostały wybrane, ze względu na najniższy koszt
    visited = set()

    current_time = start_time

    while visit_queue:
    
        cost_of_s_to_u, _, u = heappop(visit_queue)

        if u == d:
            break

        if u in visited:
            continue

        visited.add(u)
        
        neighbors = u.edges if u.name in graph.nodes else None

        if not neighbors:
            continue

        _, prev_e, prev_id, _ = predecessors[u]

        if(prev_id):
            current_time = prev_e.travel_times[prev_id][1]

        for v in neighbors:
            if v in visited:
                continue

            e : Edge = neighbors[v]
            id = e.find_nearest_departure_index2(current_time)

            if not id:
                continue
            
            cost_of_e = 0

            if cost_fun:
                cost = cost_fun(u, v, e, id, prev_e, prev_id, current_time)
                cost_of_e += cost

            if cost_fun2:
                cost = cost_fun2(u, v, e, id, prev_e, prev_id, current_time)
                cost_of_e += cost

            if cost_fun3:
                cost = cost_fun3(u, v, e, id, prev_e, prev_id, current_time)
                cost_of_e += cost

            # Koszt od s do u oraz nowo obliczony koszt od u do v
            cost_of_s_to_u_plus_cost_of_e = cost_of_s_to_u + cost_of_e

            if heuristic_func:
                additional_cost = int(heuristic_func(u, v, e, id, prev_e, prev_id, current_time))
                cost_of_s_to_u_plus_cost_of_e += additional_cost

            if v not in costs or costs[v] > cost_of_s_to_u_plus_cost_of_e:
                costs[v] = cost_of_s_to_u_plus_cost_of_e
                predecessors[v] = (u, e, id, cost_of_e)
                heappush(visit_queue, (cost_of_s_to_u_plus_cost_of_e, next(counter), v))

    if d is not None and d not in costs:
        print(f"Could not find a path from {s} to {d}")
        return

    return predecessors


def print_shortest_path_from_predecessor(predecessors, start_time, d, debug = False):
    nodes = [d]
    edges_id = []
    costs = []
    u, e, id, cost = predecessors[d]
    arrival_time = e.travel_times[id][1]
    while u is not None:
        nodes.append(u)
        edges_id.append((e, id))
        costs.append(cost)
        u, e, id, cost = predecessors[u]
    nodes.reverse()
    edges_id.reverse()
    costs.reverse()
    total_cost = sum(costs)

    if(debug):
        debug_print_path(edges_id, costs, total_cost, start_time, arrival_time)
    else:
        print_path(edges_id, costs, total_cost, start_time, arrival_time)

def debug_print_path(edges_id, costs, total_cost, start_time, arrival_time):
    for i, edge_id in enumerate(edges_id):
        print(f"Line {edge_id[0].line}: {edge_id[0].start_node.name} ({edge_id[0].travel_times[edge_id[1]][0]}) -> {edge_id[0].end_node.name} ({edge_id[0].travel_times[edge_id[1]][1]}) cost={costs[i]}")

    print(f"Total cost: {total_cost}")
    print(f"Total time: {arrival_time - start_time}")

def print_path(edges_id, costs, total_cost, start_time, arrival_time):
    prev_line = None
    prev_edge_id = None

    transfers_count = 0

    for i, edge_id in enumerate(edges_id):
        if(prev_line != edge_id[0].line):
            if (prev_line):
                print(f" -> {prev_edge_id[0].end_node.name} ({prev_edge_id[0].travel_times[prev_edge_id[1]][1]})")
            print(f"Line {edge_id[0].line}: {edge_id[0].start_node.name} ({edge_id[0].travel_times[edge_id[1]][0]})", end="")
            transfers_count+=1
        if (i == len(edges_id)-1):
            print(f" -> {edge_id[0].end_node.name} ({edge_id[0].travel_times[edge_id[1]][1]})")
        
        prev_edge_id = edge_id
        prev_line = edge_id[0].line
    print(f"\nTotal cost: {total_cost}", file=sys.stderr)
    print(f"Total time: {arrival_time - start_time}", file=sys.stderr)
    print(f"Total stops: {len(edges_id)}", file=sys.stderr)
    print(f"Total transfers: {transfers_count-1}", file=sys.stderr)

def find_path(graph, s : Node, d : Node, start_time, cost_fun = None, cost_fun2 = None, cost_fun3 = None, heuristic_func=None, debug = False):
    predecessors = shortest_path_algorithm(graph, s, d, start_time, cost_fun, cost_fun2, cost_fun3, heuristic_func=heuristic_func)
    if predecessors:
        print_shortest_path_from_predecessor(predecessors, start_time, d, debug)

## Funkcje kosztu oraz heurystyczne

Jak już wcześniej wspomniałem metoda dzięki parametryzacji jest bardzo elastyczna, a to pozwala na używanie różnych funkcji. Poniżej przykładowe funkcje:

In [8]:
def time_cost_func(u, v, e, id, prev_e, prev_id, current_time):
    cost_of_e = e.travel_times[id][1] - current_time
    if cost_of_e < timedelta():
        cost_of_e += timedelta(days=1)

    return cost_of_e.seconds

def transfer_cost_func(u, v, e, id, prev_e, prev_id, current_time):
    cost_of_e = 0 if prev_e and (e.line == prev_e.line) else 400
    return cost_of_e


def heuristic_func(u, v, e, id, prev_e, prev_id, current_time):
    return e.distance / 6

Funkcja kosztu czasu oblicza różnicę między obecnym czasem a czasem dojazdu na dany przystanek w `sekundach`.
Funkcja kosztu przesiadek nalicza wybrany koszt jeżeli linia została zmieniona. Wartość tego kosztu ma jedynie znaczenie w A*, a jej dobranie jest opisane poniżej.

### A* - funkcja heurystyczna
Jeżeli chodzi o funkcję heurystyczną, która jest wykorzystywana w algorytmie A*, to polega ona na prostym podzieleniu `odległości` pomiędzy dwoma przystankami przez przyjętą przeze mnie `średnią prędkość` komunikacji miejskiej zaokrągloną w górę. Obliczyłem ją następująco:

In [9]:
v_sum = 0
i_sum = 0

for edge in graph.edges:
    dis = edge.distance
    time = (edge.travel_times[0][1] - edge.travel_times[0][0]).seconds

    if not (dis == 0 or time == 0):
        v_sum += dis/time
        i_sum += 1

avg_velocity = v_sum/i_sum
print(f'{avg_velocity} m/s')

avg_velocity = round(avg_velocity)
print(f'{avg_velocity} m/s')

5.6723506771339505 m/s
6 m/s


Dobór prawidłowej wartości prędkości wpływa na wydajność algorytmu i sprawia, że kierunek przeszukiwania grafu jest popychany w prawidłową stronę. Wybranie perfekcyjnej wartości jest raczej nieosiągalne i uzależnione od innych zmiennych wpływających na wagę danego połączenia.

### Metody pozwalające uruchomić konkretny rodzaj algorytmu wyszukiwania
Możemy zauważyć, że tworzenie konkretnego algorytmu zoptymalizowanego pod wybrane kryterium jest bardzo proste:

In [10]:
def run_dijkstra_time(start_stop, end_stop, start_time):
    find_path(graph, start_stop, end_stop, start_time, cost_fun=time_cost_func)

def run_dijkstra_transfers(start_stop, end_stop, start_time):
    find_path(graph, start_stop, end_stop, start_time, cost_fun=transfer_cost_func)

def run_a_star_time(start_stop, end_stop, start_time):
    find_path(graph, start_stop, end_stop, start_time, cost_fun=time_cost_func, heuristic_func=heuristic_func)

def run_a_star_transfers(start_stop, end_stop, start_time):
    find_path(graph, start_stop, end_stop, start_time, cost_fun=transfer_cost_func, heuristic_func=heuristic_func)

---

# Zadanie 1

### Dane testowe
Do przetestowania poszczególnych algorytmów wykorzystałem poniższe dane:

In [11]:
start_stop = graph.nodes["Starościńska"]
end_stop = graph.nodes["Samotworska"]
start_time = base_date + timedelta(hours = 11,minutes= 24, seconds= 0)

distance(start_stop, end_stop)

14646.190246227028

## Podpunkt (a) - Algorytm Dijksty w oparciu o kryterium czasu

In [12]:
run_dijkstra_time(start_stop, end_stop, start_time)

shortest_path_algorithm() executed in 0.584910s

Line 130: Starościńska (2024-03-08 11:27:00) -> Kamieńskiego (Szpital) (2024-03-08 11:31:00)
Line K: Kamieńskiego (Szpital) (2024-03-08 11:31:00) -> Jutrosińska (2024-03-08 11:31:00)
Line 130: Jutrosińska (2024-03-08 11:31:00) -> Mochnackiego (2024-03-08 11:33:00)
Line K: Mochnackiego (2024-03-08 11:33:00) -> Kamieńskiego (2024-03-08 11:35:00)
Line D: Kamieńskiego (2024-03-08 11:35:00) -> Broniewskiego (2024-03-08 11:37:00)
Line 1: Broniewskiego (2024-03-08 11:37:00) -> DWORZEC NADODRZE (2024-03-08 11:42:00)
Line 3: DWORZEC NADODRZE (2024-03-08 11:42:00) -> Dubois (2024-03-08 11:46:00)
Line 13: Dubois (2024-03-08 11:46:00) -> PL. JANA PAWŁA II (2024-03-08 11:53:00)
Line 3: PL. JANA PAWŁA II (2024-03-08 11:55:00) -> Młodych Techników (2024-03-08 11:57:00)
Line 132: Młodych Techników (2024-03-08 11:57:00) -> Śrubowa (2024-03-08 12:00:00)
Line 13: Śrubowa (2024-03-08 12:00:00) -> Strzegomska (krzyżówka) (2024-03-08 12:08:00)
Line 107: Strze


Total cost: 5760
Total time: 1:36:00
Total stops: 43
Total transfers: 15


Egzekucja algorytmu trwała ok. 0.35s. Widzimy również cały harmonogram przejazdu - w kolejnych liniach informacje o kolejno wykorzystanych liniach komunikacyjnych (nazwa linii, czas i przystanek, na którym wsiadamy do danej linii komunikacyjnej oraz czas i przystanek, na którym kończymy korzystać z danej linii). Na dole znajują się również statystki podanej drogi.

## Podpunkt (b) - Algorytm A* w oparciu o kryterium czasu

In [13]:
run_a_star_time(start_stop, end_stop, start_time)

shortest_path_algorithm() executed in 0.587920s

Line 130: Starościńska (2024-03-08 11:27:00) -> Kamieńskiego (Szpital) (2024-03-08 11:31:00)
Line K: Kamieńskiego (Szpital) (2024-03-08 11:31:00) -> Jutrosińska (2024-03-08 11:31:00)
Line 130: Jutrosińska (2024-03-08 11:31:00) -> Mochnackiego (2024-03-08 11:33:00)
Line K: Mochnackiego (2024-03-08 11:33:00) -> Kamieńskiego (2024-03-08 11:35:00)
Line D: Kamieńskiego (2024-03-08 11:35:00) -> Broniewskiego (2024-03-08 11:37:00)
Line 1: Broniewskiego (2024-03-08 11:37:00) -> DWORZEC NADODRZE (2024-03-08 11:42:00)
Line 3: DWORZEC NADODRZE (2024-03-08 11:42:00) -> Dubois (2024-03-08 11:46:00)
Line 13: Dubois (2024-03-08 11:46:00) -> PL. JANA PAWŁA II (2024-03-08 11:53:00)
Line 3: PL. JANA PAWŁA II (2024-03-08 11:55:00) -> Młodych Techników (2024-03-08 11:57:00)
Line 132: Młodych Techników (2024-03-08 11:57:00) -> Śrubowa (2024-03-08 12:00:00)
Line 13: Śrubowa (2024-03-08 12:00:00) -> Strzegomska (krzyżówka) (2024-03-08 12:08:00)
Line 107: Strze


Total cost: 5760
Total time: 1:36:00
Total stops: 43
Total transfers: 15


Algorytm ten wykonuje się średnio o 0.02s szybciej od Dijkstry, przetestowałem różne funcje heurystyczne, ale głównie ta, którą wykorzystuję, daje najlepsze rezultaty. 

## Podpunkt (c) - Algorytm A* w oparciu o kryterium przesiadek

In [14]:
run_a_star_transfers(start_stop, end_stop, start_time)

shortest_path_algorithm() executed in 0.639074s

Line 130: Starościńska (2024-03-08 11:27:00) -> Kamieńskiego (Szpital) (2024-03-08 11:31:00)
Line K: Kamieńskiego (Szpital) (2024-03-08 11:31:00) -> Kamieńskiego (2024-03-08 12:02:00)
Line 319: Kamieńskiego (2024-03-08 14:27:00) -> Bałtycka (2024-03-08 14:28:00)
Line D: Bałtycka (2024-03-08 14:50:00) -> Rynek (2024-03-08 15:03:00)
Line 3: Rynek (2024-03-08 15:03:00) -> LEŚNICA (2024-03-08 15:37:00)
Line 117: LEŚNICA (2024-03-08 15:40:00) -> Kośnego (Jerzmanowska) (2024-03-08 16:40:00)
Line 142: Kośnego (Jerzmanowska) (2024-03-08 16:41:00) -> Jarnołtowska (Samotworska) (2024-03-08 16:43:00)
Line 909: Jarnołtowska (Samotworska) (2024-03-08 16:59:00) -> Samotworska (2024-03-08 17:00:00)



Total cost: 3200
Total time: 5:36:00
Total stops: 50
Total transfers: 7


Czas wykonywania nieco wzrósł względem kryterium czasu, ale wciąż daje zadowalające wyniki. Na tym przypadku możemy przetestować różne funkcje heurystyczne, które dadzą nam odpowiedź dlaczego wybrałem średnią prędkość komunikacji jako 6m/s.

In [15]:
def heuristic_func(u, v, e, id, prev_e, prev_id, current_time):
    return e.distance / 7 # m/s

find_path(graph, start_stop, end_stop, start_time, cost_fun=transfer_cost_func, heuristic_func=heuristic_func)

shortest_path_algorithm() executed in 0.614345s

Line 130: Starościńska (2024-03-08 11:27:00) -> Kamieńskiego (Szpital) (2024-03-08 11:31:00)
Line K: Kamieńskiego (Szpital) (2024-03-08 11:31:00) -> Kamieńskiego (2024-03-08 12:02:00)
Line D: Kamieńskiego (2024-03-08 12:02:00) -> Zajezdnia Obornicka (2024-03-08 18:13:00)
Line 101: Zajezdnia Obornicka (2024-03-08 18:15:00) -> Kwiska (2024-03-08 18:28:00)
Line 3: Kwiska (2024-03-08 18:29:00) -> LEŚNICA (2024-03-08 18:50:00)
Line 117: LEŚNICA (2024-03-08 18:55:00) -> Kośnego (Jerzmanowska) (2024-03-08 19:40:00)
Line 142: Kośnego (Jerzmanowska) (2024-03-08 19:43:00) -> Jarnołtowska (Samotworska) (2024-03-08 19:45:00)
Line 909: Jarnołtowska (Samotworska) (2024-03-08 20:56:00) -> Samotworska (2024-03-08 20:56:00)



Total cost: 3200
Total time: 9:32:00
Total stops: 47
Total transfers: 7


In [16]:
def heuristic_func(u, v, e, id, prev_e, prev_id, current_time):
    return e.distance / 4 # m/s

find_path(graph, start_stop, end_stop, start_time, cost_fun=transfer_cost_func, heuristic_func=heuristic_func)

shortest_path_algorithm() executed in 0.612696s

Line 130: Starościńska (2024-03-08 11:27:00) -> Kamieńskiego (Szpital) (2024-03-08 11:31:00)
Line K: Kamieńskiego (Szpital) (2024-03-08 11:31:00) -> Kamieńskiego (2024-03-08 12:02:00)
Line 319: Kamieńskiego (2024-03-08 14:27:00) -> Bałtycka (2024-03-08 14:28:00)
Line 129: Bałtycka (2024-03-08 14:34:00) -> OSOBOWICKA (Cmentarz II) (2024-03-08 14:38:00)
Line 104: OSOBOWICKA (Cmentarz II) (2024-03-08 14:38:00) -> most Milenijny (2024-03-08 14:40:00)
Line 101: most Milenijny (2024-03-08 14:41:00) -> Kwiska (2024-03-08 14:48:00)
Line 3: Kwiska (2024-03-08 14:48:00) -> LEŚNICA (2024-03-08 15:09:00)
Line 117: LEŚNICA (2024-03-08 15:15:00) -> Kośnego (Jerzmanowska) (2024-03-08 15:50:00)
Line 142: Kośnego (Jerzmanowska) (2024-03-08 15:56:00) -> Jarnołtowska (Samotworska) (2024-03-08 15:58:00)
Line 909: Jarnołtowska (Samotworska) (2024-03-08 15:59:00) -> Samotworska (2024-03-08 16:00:00)



Total cost: 4000
Total time: 4:36:00
Total stops: 46
Total transfers: 9


Jak możemy zauważyć wynik nie jest możliwe optymalny (więcej przesiadek, któtszy czas, lub mniej przesiadek, ale dużo dłuższy czas), a także czas wykonywania funkcji wzrósł. Dlatego też najoptymalniejszą prędkością wydaje się być 6m/s.

## Podpunkt (d) - Algorytm A* z modyfikacją
Jako modyfikację algorytmu A* postanowiłem połączyć kryterium czasu i przesiadek, aby możliwie bardzo go zoptymalizować. Znów elastyczność głównej metody algorytmu mi to ułatwi. Zdefiniowałem następujące funkcje kosztu i heurystykę, a następnie po kilku testach wybrałem najbardziej optymalne wartości.

In [17]:
def time_cost_func(u, v, e, id, prev_e, prev_id, current_time):
    cost_of_e = e.travel_times[id][1] - current_time
    if cost_of_e < timedelta():
        cost_of_e += timedelta(days=1)

    return cost_of_e.seconds

def transfer_cost_func(u, v, e, id, prev_e, prev_id, current_time):
    cost_of_e = 0 if prev_e and (e.line == prev_e.line) else 600
    return cost_of_e


def heuristic_func(u, v, e, id, prev_e, prev_id, current_time):
    return e.distance / 6

In [20]:
find_path(graph, start_stop, end_stop, start_time, cost_fun=time_cost_func, cost_fun2=transfer_cost_func, heuristic_func=heuristic_func)

shortest_path_algorithm() executed in 0.480701s

Line 130: Starościńska (2024-03-08 11:27:00) -> Kamieńskiego (Szpital) (2024-03-08 11:31:00)
Line K: Kamieńskiego (Szpital) (2024-03-08 11:31:00) -> Jutrosińska (2024-03-08 11:31:00)
Line 130: Jutrosińska (2024-03-08 11:31:00) -> Mochnackiego (2024-03-08 11:33:00)
Line K: Mochnackiego (2024-03-08 11:33:00) -> Kamieńskiego (2024-03-08 11:35:00)
Line D: Kamieńskiego (2024-03-08 11:35:00) -> Broniewskiego (2024-03-08 11:37:00)
Line 1: Broniewskiego (2024-03-08 11:37:00) -> DWORZEC NADODRZE (2024-03-08 11:42:00)
Line 3: DWORZEC NADODRZE (2024-03-08 11:42:00) -> Kosmonautów (Szpital) (2024-03-08 12:25:00)
Line 107: Kosmonautów (Szpital) (2024-03-08 12:43:00) -> Jerzmanowska nr 17 (2024-03-08 12:47:00)
Line 129: Jerzmanowska nr 17 (2024-03-08 12:50:00) -> Jerzmanowo (Cmentarz I) (2024-03-08 12:53:00)
Line 117: Jerzmanowo (Cmentarz I) (2024-03-08 12:53:00) -> Kośnego (Jerzmanowska) (2024-03-08 12:57:00)
Line 142: Kośnego (Jerzmanowska) (2024-03


Total cost: 12960
Total time: 1:36:00
Total stops: 43
Total transfers: 11


Koszt czasu i funkcja heurystyczna wydają mi się najbardziej zoptymalizowane, dlatego skupiłem się głównie na koszcie przesiadek. Po kilku przeprowadzonych testach wartość tą ustaliłem na 600. To z kolei wpłynęło na algorytm bardzo pozytywnie. Czas trwania algorytmu pomimo dodatkowych obliczeń nie wzrósł, a otrzymane wyniki zapewniły najlepszą możliwie drogę z minimalną ilością przesiadek oraz najkrótszym czasem podróży.

## Główny program z wczytywaniem danych ze standardowego wejścia
Napisałem poniższą metodę, która działa zgodnie z wymaganiami w instrukcji - dodałem jedynie jeszcze możliwość wyboru algorytmu.

In [21]:
def run():
    start_stop = graph.nodes[input("Podaj przystanek początkowy: ")]
    end_stop = graph.nodes[input("Podaj przystanek końcowy: ")]
    start_time_str_raw = input("Podaj czas pojawienia się na przystanku początkowym (format HH:MM:SS): ")
    start_time_str = start_time_str_raw.split(sep=":")
    start_time = base_date + timedelta(hours=int(start_time_str[0]), minutes=int(start_time_str[1]), seconds=int(start_time_str[2]))
    algorithm = input("Wybierz algorytm (d - dijkstra, a - astar): ")
    criterion = input("Podaj kryterium optymalizacji (t - czas, p - liczba przesiadek): ")

    print(f'{algorithm}-{criterion}: {(start_time_str_raw)} {start_stop.name} -> {end_stop.name}')


    if criterion == "t":
        if algorithm == "d":
            run_dijkstra_time(start_stop, end_stop, start_time)
        elif algorithm == "a":
            run_a_star_time(start_stop, end_stop, start_time)
        else: 
            print("Podano nieprawidłowy algorytm!")
    elif criterion == "p":
        if algorithm == "d":
            run_dijkstra_transfers(start_stop, end_stop, start_time)
        elif algorithm == "a":
            run_a_star_transfers(start_stop, end_stop, start_time)
        else: 
            print("Podano nieprawidłowy algorytm!")
    else:
        print("Podano nieprawidłowe kryterium!")

run()

a-t: 12:23:23 Tyrmanda -> FAT
shortest_path_algorithm() executed in 0.033080s

Line 107: Tyrmanda (2024-03-08 12:25:00) -> FAT (2024-03-08 12:37:00)



Total cost: 817
Total time: 0:13:37
Total stops: 8
Total transfers: 0
